# Caso Práctico - Visualización de Datos
---
Visualización de Datos del Máster Universitario en Ingeniería y Ciencia de Datos (2023/2024)

Alumno: **Javier Orive Soto**


# 1. Importación de dependencias

In [1]:
import pandas as pd
import panel as pn
pn.extension('vega')

import numpy as np
import altair as alt

import datetime as dt

# 2. Importación de datos

## 2.1 Datos económicos

In [2]:
# Cargar el archivo Excel
#file_path = 'Datos/Economico/PrecioMedioHorarioFinal_2023_d.xls'
file_path = 'Datos/Generacion/GeneracionTotal_2023_h.csv'
#df = pd.read_excel(file_path)
df_G = pd.read_csv(file_path, delimiter=';')

# Seleccionar solo las columnas de interés
#df = df[['name', 'value', 'datetime']]

In [3]:
# Convertir la columna datetime a tipo datetime
df_G['datetime'] = pd.to_datetime(df_G['datetime'], utc = True) #, utc = True

# Convertir la columna value a tipo float
df_G['value'] = df_G['value'].astype(float)

# Convertir la columna name a tipo categoría
df_G['name'] = df_G['name'].astype('category')

# Elimino las columnas vacías
df_G = df_G.drop(['geoid', 'geoname'], axis=1)

In [4]:
df_G = df_G.drop(df_G[df_G['id']== 10195].index)

df_G['date'] = df_G['datetime'].dt.date

df_G['year'] = df_G['datetime'].dt.year
df_G['month'] = df_G['datetime'].dt.month
df_G['day'] = df_G['datetime'].dt.day
df_G['hour'] = df_G['datetime'].dt.hour

In [5]:
df_G.head()

,id,name,value,datetime,date
4321,1169,Generación medida Biogás,83.937,2022-12-31 23:00:00+00:00,2022-12-31
4322,1169,Generación medida Biogás,81.609,2023-01-01 00:00:00+00:00,2023-01-01
4323,1169,Generación medida Biogás,80.939,2023-01-01 01:00:00+00:00,2023-01-01
4324,1169,Generación medida Biogás,80.873,2023-01-01 02:00:00+00:00,2023-01-01
4325,1169,Generación medida Biogás,80.765,2023-01-01 03:00:00+00:00,2023-01-01


In [6]:
df_G.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155473 entries, 4321 to 164210
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   id        155473 non-null  int64              
 1   name      155473 non-null  category           
 2   value     155473 non-null  float64            
 3   datetime  155473 non-null  datetime64[ns, UTC]
 4   date      155473 non-null  object             
dtypes: category(1), datetime64[ns, UTC](1), float64(1), int64(1), object(1)
memory usage: 6.1+ MB


In [7]:
# https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289
date_slider = pn.widgets.DateSlider(name='Date Slider', start=dt.datetime(2023, 1, 1), end=dt.datetime(2023, 12, 31), value=dt.datetime(2023, 1, 1))

date_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'f0f74575-c6bb-4a1e-9c7e-ed035828a48a': {'version…

In [35]:
@pn.depends(date_slider.param.value)
def get_plot(date):
     # Load and format the data
     df = df_G # define df
     ##df[‘date’] = pd.to_datetime(df[‘date’])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_slider.value
     # create filter mask for the dataframe
     mask = (df['date'] == start_date)
     df = df.loc[mask] # filter the dataframe

     selection = alt.selection_point(fields=['name'], bind='legend')

     # create the Altair chart object
     chart = alt.Chart(df).mark_area(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('sum(value):Q', title='Potencia eléctrica (MW)', scale=alt.Scale(domain=[0, 42000])),  # Suma de potencia
               color=alt.Color('name:N', title='Tecnología'),  # Diferenciar por tecnología
               opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
               ).properties(
               title='Generación eléctrica por tecnología',
               width=800,
               height=400
               ).add_params(selection)

     return chart

In [36]:
title = '## Desglose de generación eléctrica'
subtitle = 'Este dashboard permite al usuario seleccionar\n el día para el que se quieren visualizar los datos.\n\n Además, también se puede seleccionar la tecnología\n para resaltarla en el gráfico.'

dashboard = pn.Row(get_plot,
                   pn.Column(title, subtitle, date_slider)
)

/usuarios/jos/.conda/envs/jos_prueba/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function get_plot at 0x7f74490608b0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function get_plot at 0x7f74490608b0> will be resolved to its underlying value alt.Chart(...). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)


In [37]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'e9ddbaf9-1ec9-40bf-acad-28ed237c4f9d': {'version…

In [49]:
df_aux = df_G.groupby('name')['value'].sum().sort_values(ascending=False).reset_index()

In [50]:
df_aux

,name,value
0,Generación medida Eólica terrestre,6.108076e+07
1,Generación medida Nuclear,5.412415e+07
2,Generación medida Ciclo combinado,3.920130e+07
3,Generación medida Solar fotovoltaica,3.648300e+07
4,Generación medida Hidráulica UGH,2.068712e+07
5,Generación medida Gas Natural Cogeneración,1.653210e+07
6,Generación medida Turbinación bombeo,5.184742e+06
7,Generación medida Solar térmica,4.673006e+06
8,Generación medida Hidráulica no UGH,4.547275e+06
9,Generación medida Hulla antracita,2.933010e+06


In [67]:
selection = alt.selection_point(fields=['name'], bind='legend')

base = alt.Chart(df_aux).encode(
    alt.Theta("sum(value):Q", sort=df_aux['name'].to_list()).stack(True),
    alt.Radius("sum(value)").scale(type="sqrt", zero=True),
    color=alt.Color("name:N", sort=alt.EncodingSortField(field="value", order="descending")),
    #color="name:N",
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection)

c = base.mark_arc(innerRadius=20, stroke="#fff")

#c2 = base.mark_text(radiusOffset=10).encode(text="name")

c

alt.Chart(...)

In [ ]:
df = df.pivot(index='datetime', columns='name', values='value').reset_index()

In [ ]:
df

In [ ]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='Precio medio horario componente banda secundaria:Q'
)

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='value:Q',
  color='name:N'
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N'
)

In [ ]:
selection = alt.selection_point(fields=['name'], bind='legend')

In [ ]:
# Plotting
chart.mark_line().encode(
    x='datetime:T',
    y='value:Q',
    tooltip=['datetime:T', 'value:Q']
).properties(
    title='Hourly Average Price Over Time',
    width=800,
    height=400
).interactive()





In [ ]:
chart.show

In [ ]:
# Crear el gráfico apilado
chart.mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()


In [ ]:
# Crear el gráfico apilado
alt.Chart(df).mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()